# Contexto:
### O site [Monitoramento COVID-19](https://www.covid-19.pa.gov.br/) administrado pela **Secretaria de Saúde Pública do Pará (SESPA)** disponibiliza uma série de informações acerca do status da epidemia no estado do Pará.

# Problema:
### Essas informações estão exibidas no site, então para utilizá-las haveria a necessidade de um parse das informações através de um webcrawler.

# Solução
### Achar as endpoints responsáveis pelo retorno das informações e abstrair os requisitos necessários para o acesso ás mesmas.

# 1. Bibliotecas Utilizadas
### 1.1 Instalação
```bash
pip install requests
pip install json
pip install base64
pip install pandas
pip install sqlalchemy
```
### 1.2 Importação

In [6]:
from base64 import b64encode
import json
import pandas as pd
import requests as rq

# 2. Definindo dados necessários ao acesso

In [7]:
# default guest user data
username = '00000000000' # default username for guest user
password = 'bc7786ggsf66a715fd847a63a2e12f2c20' # default pwd for guest user

# verify secure request?
verify = True

# 3. Definindo as rotas de acesso da API

In [8]:
# main routes
rota_raiz = 'https://www.covid-19.pa.gov.br/monitoramento-corona-service/'
rota_testes = 'testecovid/'
rota_status = 'statuscorona/'
rota_leitos = 'leitos-novos/'

# side routes
rota_serie_historica = rota_status + 'serie-historica-confirmados-obitos/'
rota_positivos_faixa_etaria = rota_status + 'casos-positivos-faixa-etaria/'
rota_confirmados_obitos_municipios = rota_status + 'casos-confirmados-obitos-por-municipio/'
rota_percentual_por_sexo = rota_status + 'percentual-por-sexo/'
rota_resumido_recentes = rota_status + 'maisrecente/'
rota_testes_recentes = rota_testes + 'maisrecente/'

# 4. Definindo o método para obter as credenciais de autenticação

In [9]:
def getAuthBearer(username, password):
    """
    Get authentication bearer to pass on further requests
    Required:
        username : username
        password: password
    Returns
        Authentication bearer as str
    """
    # login string
    string = username + ':' + password
    bytes_string = string.encode("utf-8")
    encoded_credentials = b64encode(bytes_string).decode("utf-8")
    
    # get auth bearer
    bearer = rq.post(
        url = rota_raiz + 'login/public',
        verify = verify,
        headers = {
            'Authorization': 'Basic ' + encoded_credentials
        }
    ).headers['Authorization']
        
    # return bearer
    return bearer

# 5. Definindo o método para obter os dados dos endpoints

In [10]:
def generateJsonData(route):
    """
    Request data from API routes
    Required:
        route: route to get data
    Returns
        Normalized data as json dataframe
    """
    # get data
    data = rq.get(
        url = rota_raiz + route,
        headers = {
            'Authorization': getAuthBearer(username, password)
        },
        verify = verify
    ).text
    
    # parse data as json
    parsed_data = json.loads(data)
    
    # normalize json data and save as dataframe
    normalized_data = pd.json_normalize(parsed_data)
    
    # return data as json parsed dataframe
    return normalized_data

# 6. Populando variáveis

In [12]:
num_testes = generateJsonData(rota_testes_recentes)
serie_historica = generateJsonData(rota_serie_historica)
atualizacoes_serie_historica = generateJsonData(rota_status)
positivos_faixa_etaria = generateJsonData(rota_positivos_faixa_etaria)
confirmados_obitos_municipios = generateJsonData(rota_confirmados_obitos_municipios)
percentual_por_sexo = generateJsonData(rota_percentual_por_sexo)
resumido_recentes = generateJsonData(rota_resumido_recentes)
leitos_novos = generateJsonData(rota_leitos)

/usr/local/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

## O projeto ainda está em andamento.

## A ser implementado:
### - Exportar os dados diretamente para bancos de dados